# Indice espectrales NDVI en GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [7]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 10)

In [6]:
Map.addLayer(zona_ee, {"color" : "red"}, name = "Zona")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Seleccionar Imagenes Landsat RS

In [8]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
                  .select(["B2","B3","B4","B5","B6","B7"],
                            ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .reproject(crs = "EPSG:32719", scale = 30)

In [9]:
# Seleccionar una imagen Sentinel2
img_S2 = ee.Image("COPERNICUS/S2_SR/20190808T145729_20190808T145732_T19LCF")\
                  .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],
                        ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001) # 0 - 1

In [10]:
# Nombre de las bandas
print("Nombre bandas L8: ",img_L8.bandNames().getInfo())
print("Nombre bandas S2: ",img_S2.bandNames().getInfo())

Nombre bandas L8:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Nombre bandas S2:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [11]:
# Consultar Proyeccion
print("Proyeccion L8: ",img_L8.projection().getInfo())
print("Proyeccion S2: ",img_S2.select(['BLUE']).projection().getInfo())

Proyeccion L8:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion S2:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [10, 0, 300000, 0, -10, 8600020]}


In [12]:
# Simbologia estala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

## Recortar imagen zona

In [13]:
# Recortar informacion con la zona de estudio
img_L8_clip = img_L8.clip(zona_ee)
img_S2_clip = img_S2.clip(zona_ee)

In [14]:
# Visualizar 
Map.addLayer(img_L8_clip, viz_es, name = "Landsat8 RS")
Map.addLayer(img_S2_clip, viz_es, name = "Sentinel2 RS")
Map

Map(bottom=140852.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Determinar NDVI

### Forma 1 NDVI

In [15]:
B_NIR = img_L8_clip.select('NIR')
B_RED = img_L8_clip.select('RED')

In [16]:
# Determinar NDVI forma 1 Landsat8
ndvi1 = B_NIR.subtract(B_RED)\
             .divide(B_NIR.add(B_RED)).rename("NDVI")

In [17]:
type(ndvi1)

ee.image.Image

In [18]:
ndvi1.bandNames().getInfo()

['NDVI']

### Forma 2 NDVI

In [19]:
# Determinar NDVI forma 2 Landsat8
ndvi2 = img_L8_clip.normalizedDifference(['NIR','RED']).rename("NDVI")

### Forma 3 NDVI

In [20]:
# Determinar NDVI forma 3 Funcion GEE
def Calc_NDVI(image):
    ndvi = image.expression('float(nir - red)/(nir + red)', {
        'nir': image.select('NIR'),
        'red': image.select('RED')
    })

    return ndvi.rename("NDVI")

In [21]:
# Calcular indices NDVI con funcion
ndvi_L8 = Calc_NDVI(img_L8_clip)
ndvi_S2 = Calc_NDVI(img_S2_clip)

## Visualizar Mapa NDVI

In [22]:
# Simbologia de NDVI
viz_ndvi = {
    'palette': ['FFFFFF', 'CE7E45','DF923D', 'F1B555',
                'FCD163', '99B718','74A901', '66A000',
                '529400', '3E8601','207401', '056201',
                '004C00', '023B01','012E01', '011D01',
                '011301'],
    'min': -0.4,
    'max': 1,
    'bands': "NDVI"
}

In [23]:
# Visualizar 
Map.addLayer(ndvi_L8, viz_ndvi, name = "Landsat8 NDVI")
Map.addLayer(ndvi_S2, viz_ndvi, name = "Sentinel2 NDVI")
Map

Map(bottom=562578.0, center=[-12.931622281128998, -70.07200241088869], controls=(WidgetControl(options=['posit…

## Clasificacion de NDVI 
- NDVI < 0.2  = 1
- NDVI >= 0.2  = 2
- NDVI >= 0.65 = 3

In [24]:
L8_clas_ndvi = ee.Image(0).where(ndvi_L8.lt(0.2),1)\
                      .where(ndvi_L8.gte(0.2),2)\
                      .where(ndvi_L8.gte(0.65),3)\
                      .clip(zona_ee)

In [32]:
# Simbologia de clasificacion de NDVI 3 clases
viz_clas = {
    'palette': ["#ff03e0",'#fffa53',"#239460"],
    'min': 1,
    'max': 3
}

In [33]:
# Visualizar 
Map.addLayer(L8_clas_ndvi , viz_clas, name = "L8 NDVI Class")
Map.add_colorbar(viz_clas, label="Clase NDVI", layer_name="NDVI")
Map

Map(bottom=281404.0, center=[-12.90819810831852, -69.90703582763673], controls=(WidgetControl(options=['positi…

## Exportar Imagen

In [27]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [28]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [29]:
# Exportar imagen directorio
geemap.ee_export_image(L8_clas_ndvi, filename="L8_clas_ndvi.tif", scale=30, region=geometria, file_per_band=False)
geemap.ee_export_image(ndvi_L8, filename="L8_ndvi.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_clas_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_ndvi.tif


In [31]:
geemap.ee_export_image_to_drive(ndvi_S2, description='S2_ndvi', folder='GEE_Python', region=geometria, scale=10)

Exporting S2_ndvi ...
